<h1  style='color:white; background:#8D8F8A; border:0;text-align: center' class="list-group-item list-group-item-action active">KANNADA MNIST CHARACTER RECOGNITION </h1><a id = "1" ></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='color:white; background:#8D8F8A; border:0' role="tab" aria-controls="home"><center>Quick Navigation</center></h3>

* [Importing Libraries](#1.1)
* [Data Preprocessing](#2)
    - [Load Training Dataset](#2.1)
    - [Split Data](#2.2)
    - [Image Preprocessing](#2.3)
* [Data Visualization](#3)
* [Activation function](#4)
* [Model Summary](#5)
* [Data Augmentation](#6)
* [Save and Load Model](#6.2)
* [Plot Accuracy and Loss curve](#7)
* [Evaluation](#8)

<h1  style='color:white; background:#8D8F8A; border:0' class="list-group-item list-group-item-action active">IMPORTING LIBRARIES</h1><a id = "1.1" ></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import keras
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout,MaxPool2D, LSTM, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import ELU,Activation
from tensorflow.keras.losses import sparse_categorical_crossentropy, categorical_crossentropy, binary_crossentropy

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<h1 style='color:white; background:#8D8F8A; border:0;text-align: center' class="list-group-item list-group-item-action active">2. DATA PREPROCESSING</h1><a id = "2" ></a>

<h2  style='color:white; background:#8D8F8A; border:0;text-align: center' class="list-group-item list-group-item-action active">2.1 LOAD TRAINING DATASET</h2><a id = "2.1" ></a>

In [ ]:
train_data = pd.read_csv("../input/Kannada-MNIST/train.csv")

In [ ]:
test_file = "../input/Kannada-MNIST/test.csv"
test_data = pd.read_csv(test_file)

In [ ]:
val_data = pd.read_csv('../input/Kannada-MNIST/Dig-MNIST.csv')

**Shape**

In [ ]:
print(f"train.csv size is {train_data.shape}")
print(f"test.csv size is {test_data.shape}")
print(f"val_data size is {val_data.shape}")

<h2  style='color:white; background:#8D8F8A; border:0;text-align: center' class="list-group-item list-group-item-action active">2.2 SPLIT DATA</h2><a id = "2.2" ></a>

In [ ]:
img_rows, img_cols = 28,28
num_classes = 10

<h2  style='color:white; background:#8D8F8A; border:0;text-align: center' class="list-group-item list-group-item-action active">2.3 PREPROCESSING</h2><a id = "2.3" ></a>


**ONE HOT ENCODING:**

One hot encoding is one method of converting data to prepare it for an algorithm and get a better prediction. With one-hot, we convert each categorical value into a new categorical column and assign a binary value of 1 or 0 to those columns. Each integer value is represented as a binary vector.

In [ ]:
def data_prep(raw):
    out_y = tensorflow.keras.utils.to_categorical(raw.label, num_classes)

    num_images = raw.shape[0]
    x_as_array = raw.values[:,1:]
    x_shaped_array = x_as_array.reshape(num_images, img_rows, img_cols, 1)
    # normalization
    out_x = x_shaped_array / 255
    return out_x, out_y

x_train, y_train = data_prep(train_data)
x_val,y_val = data_prep(val_data)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(np.concatenate((x_train, x_val)),np.concatenate((y_train, y_val)),
                                                   test_size=0.1,shuffle=True)

**Data Dimension**

In [ ]:
print(f"Training data size is {x_train.shape}")
print(f"Training data size is {y_train.shape}")
print(f"Testing data size is {x_test.shape}")
print(f"Training data size is {y_test.shape}")

<h1  style='color:white; background:#8D8F8A; border:0;text-align: center' class="list-group-item list-group-item-action active">3. VISUALIZE DATA</h1><a id = "3" ></a>

In [ ]:
def get_image(row, dataset):
    row_values = dataset.iloc[row].values
    image_matrix = row_values.reshape(img_rows, img_cols)
    return image_matrix

fig, ax = plt.subplots(1, 5, figsize=(12, 9), dpi=120)
plt.setp(ax, xticks=[], yticks=[])

ax_n=0
for i in range(1,6,1):
    ax[ax_n].imshow(get_image(i, train_data.drop('label',axis=1)))
    ax_n+=1

In [ ]:
title=[]
for i in range(1, 10):
    for j in range(0,10):
        if y_train[i][j] == 1:
            title.append(j)
title

In [ ]:
plt.figure(figsize=(7,9))
for i in range(1, 10):
    plt.subplot(330 + i)
    plt.imshow(x_train[i], cmap=plt.get_cmap('Greys'))
    plt.title(title[i-1])
    plt.axis('off')
plt.tight_layout()

<h1  style='color:white; background:#8D8F8A; border:0;text-align: center' class="list-group-item list-group-item-action active">4. ACTIVATION FUNCTION</h1><a id = "4" ></a>


- **LeakyReLu**

In [ ]:
leakyrelu = tensorflow.keras.layers.LeakyReLU(alpha=0.01)

<h2  style='color:white; background:#8D8F8A; border:0;text-align: center' class="list-group-item list-group-item-action active">5. MODEL SUMMARY</h2><a id = "5" ></a>


In [ ]:
model = Sequential()

model.add(Conv2D(64, kernel_size=3, padding='same',input_shape=(img_rows, img_cols, 1)))
model.add(BatchNormalization(scale=False, center=True))
model.add(Activation('relu'))
model.add(Conv2D(64, kernel_size=3, padding = 'same'))
model.add(BatchNormalization(scale=False, center=True))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, kernel_size=(3,3), padding = 'same'))
model.add(BatchNormalization(scale=False,center=True))
model.add(Activation('relu'))
model.add(Conv2D(128, kernel_size=(3, 3), padding = 'same'))
model.add(BatchNormalization(scale=False,center=True))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(256))
model.add(BatchNormalization(scale=False,center=True))
model.add(Activation('relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()


In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)
from IPython.display import Image
Image("model.png")

<h1  style='color:white; background:#8D8F8A; border:0;text-align: center' class="list-group-item list-group-item-action active">6. DATA AUGMENTATION</h1><a id = "6" ></a>

One way to avoid overfitting and improve the accuracy is to increase the variability of existing samples. Which is also helps to compensate lack of data. Data augmentation generates data from existing samples by applying various transformations to the original dataset. This method aims to increase the number of unique input samples, which, in turn, will allow the model to show better accuracy on the validation dataset.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# use data augmentation to improve accuracy and prevent overfitting
augs_gen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False, 
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=10,  
        zoom_range = 0.2, 
        width_shift_range=0.2,  
        height_shift_range=0.2, 
        horizontal_flip=False,  
        vertical_flip=False) 

train_generator = augs_gen.flow(x_train, y_train, batch_size=128)
valid_generator = augs_gen.flow(x_val, y_val, batch_size=128)

<h2  style='color:white; background:#8D8F8A; border:0;text-align: center' class="list-group-item list-group-item-action active"> CallBacks</h2><a id = "6.1" ></a>

**EarlyStopping**

In [ ]:
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_accuracy', min_delta=0.00001, patience=6, mode='auto', restore_best_weights=True)

**Learning Rate Reduction**

In [ ]:
lr_reduction = ReduceLROnPlateau(monitor='val_accuracy', min_lr=0.0001 ,patience=6, verbose=1,  factor=0.6)

In [ ]:
epochs = 25
BATCH = 128

In [ ]:
model.compile(optimizer = 'adam',loss = binary_crossentropy,metrics=['accuracy'])
model_fit = model.fit(train_generator, epochs=epochs,batch_size =BATCH ,validation_data= valid_generator, verbose =1,callbacks=[early_stop,lr_reduction])

<h2  style='color:white; background:#8D8F8A; border:0;text-align: center' class="list-group-item list-group-item-action active">SAVE MODEL</h2><a id = "6.2" ></a>

In [ ]:
!mkdir -p saved_model
model.save('saved_model/my_model')

In [ ]:
load_model = tensorflow.keras.models.load_model('saved_model/my_model')
load_model.summary()

<h2  style='color:white; background:#8D8F8A; border:0;text-align: center' class="list-group-item list-group-item-action active">7. PLOT ACCURACY AND LOSS CURVE</h2><a id = "7" ></a>

In [ ]:
# Defining Figure
f = plt.figure(figsize=(20,7))

#For Accuracy - subplot
f.add_subplot(121)

plt.plot(model_fit.epoch,model_fit.history['accuracy'],label = "training accuracy") # Accuracy curve for training set
plt.plot(model_fit.epoch,model_fit.history['val_accuracy'],label = "validation accuracy")

plt.title("Accuracy Curve",fontsize=18)
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("Accuracy",fontsize=15)
plt.grid(alpha=0.3)
plt.legend()

In [ ]:
#Adding Subplot (For Loss)
f.add_subplot(122)

plt.plot(model_fit.epoch,model_fit.history['loss'],label="training loss") # Loss curve for training set
plt.plot(model_fit.epoch,model_fit.history['val_loss'],label="validation loss")

plt.title("Loss Curve",fontsize=18)
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("Loss",fontsize=15)
plt.grid(alpha=0.3)
plt.legend()

plt.show()


<h1  style='color:white; background:#8D8F8A; border:0;text-align: center' class="list-group-item list-group-item-action active">8. EVALUATION</h1><a id = "8" ></a>

In [ ]:
evaluate_test = model.evaluate(x_test, y_test, verbose=1)

print("\nAccuracy =", "{:.7f}%".format(evaluate_test[1]*100))
print("Loss     =" ,"{:.9f}".format(evaluate_test[0]))

In [ ]:
y_predict = model.predict(x_test)

In [ ]:
y_predict_max = np.argmax(y_predict,axis=1) 
y_predict_max


<h2  style='color:white; background:#8D8F8A; border:0;text-align: center' class="list-group-item list-group-item-action active">8.1 SUBMISSION</h2><a id = "8.1" ></a>

**Test Data Preprocessing**

In [ ]:
def data_prep(raw):
    num_images = raw.shape[0]
    x_as_array = raw.values[:,1:]
    x_shaped_array = x_as_array.reshape(num_images, img_rows, img_cols, 1)
    
    return x_shaped_array

test_x = data_prep(test_data)
test_x.shape

In [ ]:
test_generator = augs_gen.flow(test_x)

In [ ]:
y_hat = model.predict_generator(test_generator)

In [ ]:
submission_label = np.argmax(y_hat, axis=1)
submission_label = pd.Series(submission_label, name="label")

image_id = pd.Series(range(0,len(test_data)))
image_id = pd.Series(image_id, name="id")

In [ ]:
submission = pd.concat([image_id,submission_label],axis = 1)
submission.to_csv("submission.csv", index=False)
pd.read_csv("submission.csv").head()